In [1]:
!pip3 install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 549.1 kB/s eta 0:00:00a 0:00:01


In [2]:
import psycopg2
import psycopg2.extras as extras
import re

In [3]:
PSQL_HOST = 'localhost'
PSQL_PORT = 5432
PSQL_DB = 'database'
PSQL_USER = 'user'
PSQL_PWD = 'password'

In [4]:
conn = psycopg2.connect(host=PSQL_HOST, dbname=PSQL_DB, user=PSQL_USER, password=PSQL_PWD, port=PSQL_PORT, connect_timeout=5)
cursor = conn.cursor()

In [5]:
cursor.close()

In [ ]:
class Point:
    def __init__(self,id_point, geom_point):
        self.id_point = id_point
        self.geom_point = geom_point

class Line:
    def __init__(self,id_line, geom_line, cost):
        self.id_point = id_line
        self.geom_point = geom_line
        self.cost = cost

In [ ]:
def change_cost(table_name, value):
    # Database connection details
    conn = psycopg2.connect(host=PSQL_HOST, dbname=PSQL_DB, user=PSQL_USER, password=PSQL_PWD, port=PSQL_PORT, connect_timeout=5)

    cursor = conn.cursor()

    # Parameters
    column_name = "peso"

    # SQL query to update all rows
    sql_update_query = f"""
    UPDATE {table_name}
    SET {column_name} = %s;
    """

    # Execute the query and pass the value
    cursor.execute(sql_update_query, (value,))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    print(f"All rows in column '{column_name}' of table '{table_name}' have been set to {value}.")

change_cost('eps.red2', 5)
change_cost('eps.red1', 3)

All rows in column 'peso' of table 'eps.red2' have been set to 5.
All rows in column 'peso' of table 'eps.red1' have been set to 3.


In [25]:
def get_lon_lat(cursor, table, id):
    sql_query = f'SELECT ST_AsText(geom) from {table} WHERE id = {id}'
    cursor.execute(sql_query)
    result = cursor.fetchone()[0]
    match = re.match(r'POINT\(([\d\-.]+) ([\d\-.]+)\)', result)
    return float(match.group(1)), float(match.group(2))

In [ ]:
def get_closest_point(lon, lat):

    # SQL query to find the closest point
    sql_query = """
    SELECT id, ST_AsText(geom), ST_Distance(geom, ST_MakePoint(%s, %s)::geography) AS distance
    FROM places
    ORDER BY ST_Distance(geom, ST_MakePoint(%s, %s)::geography)
    LIMIT 1;
    """
    
    cursor.execute(sql_query, (lon, lat, lon, lat))

    result = cursor.fetchone()
    
    return result

# Example usage
conn = psycopg2.connect(
        host=PSQL_HOST,
        dbname=PSQL_DB,
        user=PSQL_USER,
        password=PSQL_PWD,
        port=PSQL_PORT
    )
cursor = conn.cursor()
A_x, A_y = get_lon_lat(cursor, 'eps.inicio', 1)
B_x, B_y = get_lon_lat(cursor, 'eps.final', 6)
#closest_point = get_closest_point(-75.123456, 40.123456)
#print("Closest Point:", closest_point)
print(A_x, A_y)
cursor.close()
conn.close()
    

-5.300087566340136 37.588388615274326
